Code for Liang, Y., Huang, G., Zhao, Z. (2022). Joint demand prediction for multimodal systems: A multi-task multi-relational spatiotemporal graph neural network approach. Transportation Research Part C: Emerging Technologies, 140, 103731.

Part of the code comes from STGCN (https://github.com/hazdzz/STGCN).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torch.autograd as autograd
import math
import time
import pickle
from tqdm import tqdm
import numpy as np
from scipy import sparse

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Adj Processor

In [ ]:
import scipy.sparse as sp
import numpy as np
from scipy.linalg import eigvalsh
from scipy.linalg import fractional_matrix_power

def calculate_random_walk_matrix(adj_mx):
    """
    Returns the random walk adjacency matrix. This is for D_GCN
    """
    adj_mx = sp.coo_matrix(adj_mx)
    d = np.array(adj_mx.sum(1))+(1e-5)
    d_inv = np.power(d, -1).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat_inv = sp.diags(d_inv)
    random_walk_mx = d_mat_inv.dot(adj_mx).tocoo()
    return random_walk_mx.toarray()

def calculate_laplacian_matrix(adj_mat, mat_type):
    n_vertex = adj_mat.shape[0]
    id_mat = np.asmatrix(np.identity(n_vertex))

    # D_row
    deg_mat_row = np.asmatrix(np.diag(np.sum(adj_mat, axis=1)))
    # D_com
    #deg_mat_col = np.asmatrix(np.diag(np.sum(adj_mat, axis=0)))

    # D = D_row as default
    deg_mat = deg_mat_row
    adj_mat = np.asmatrix(adj_mat)

    # wid_A = A + I
    wid_adj_mat = adj_mat + id_mat
    # wid_D = D + I
    wid_deg_mat = deg_mat + id_mat

    # Combinatorial Laplacian
    # L_com = D - A
    com_lap_mat = deg_mat - adj_mat

    if mat_type == 'id_mat':
        return id_mat
    elif mat_type == 'com_lap_mat':
        return com_lap_mat

    if (mat_type == 'sym_normd_lap_mat') or (mat_type == 'wid_sym_normd_lap_mat') or (mat_type == 'hat_sym_normd_lap_mat'):
        deg_mat_inv_sqrt = fractional_matrix_power(deg_mat, -0.5)
        deg_mat_inv_sqrt[np.isinf(deg_mat_inv_sqrt)] = 0.

        wid_deg_mat_inv_sqrt = fractional_matrix_power(wid_deg_mat, -0.5)
        wid_deg_mat_inv_sqrt[np.isinf(wid_deg_mat_inv_sqrt)] = 0.

        # Symmetric normalized Laplacian
        # For SpectraConv
        # To [0, 1]
        # L_sym = D^{-0.5} * L_com * D^{-0.5} = I - D^{-0.5} * A * D^{-0.5}
        sym_normd_lap_mat = np.matmul(np.matmul(deg_mat_inv_sqrt, com_lap_mat), deg_mat_inv_sqrt)

        # For ChebConv
        # From [0, 1] to [-1, 1]
        # wid_L_sym = 2 * L_sym / lambda_max_sym - I
        #sym_max_lambda = max(np.linalg.eigvalsh(sym_normd_lap_mat))
        sym_max_lambda = max(eigvalsh(sym_normd_lap_mat))
        wid_sym_normd_lap_mat = 2 * sym_normd_lap_mat / sym_max_lambda - id_mat

        # For GCNConv
        # hat_L_sym = wid_D^{-0.5} * wid_A * wid_D^{-0.5}
        hat_sym_normd_lap_mat = np.matmul(np.matmul(wid_deg_mat_inv_sqrt, wid_adj_mat), wid_deg_mat_inv_sqrt)

        if mat_type == 'sym_normd_lap_mat':
            return sym_normd_lap_mat
        elif mat_type == 'wid_sym_normd_lap_mat':
            return wid_sym_normd_lap_mat
        elif mat_type == 'hat_sym_normd_lap_mat':
            return hat_sym_normd_lap_mat

    elif (mat_type == 'rw_normd_lap_mat') or (mat_type == 'wid_rw_normd_lap_mat') or (mat_type == 'hat_rw_normd_lap_mat'):
        try:
            # There is a small possibility that the degree matrix is a singular matrix.
            deg_mat_inv = np.linalg.inv(deg_mat)
        except:
            print(f'The degree matrix is a singular matrix. Cannot use random walk normalized Laplacian matrix.')
        else:
            deg_mat_inv[np.isinf(deg_mat_inv)] = 0.

        wid_deg_mat_inv = np.linalg.inv(wid_deg_mat)
        wid_deg_mat_inv[np.isinf(wid_deg_mat_inv)] = 0.

        # Random Walk normalized Laplacian
        # For SpectraConv
        # To [0, 1]
        # L_rw = D^{-1} * L_com = I - D^{-1} * A
        rw_normd_lap_mat = np.matmul(deg_mat_inv, com_lap_mat)

        # For ChebConv
        # From [0, 1] to [-1, 1]
        # wid_L_rw = 2 * L_rw / lambda_max_rw - I
        #rw_max_lambda = max(np.linalg.eigvalsh(rw_normd_lap_mat))
        rw_max_lambda = max(eigvalsh(rw_normd_lap_mat))
        wid_rw_normd_lap_mat = 2 * rw_normd_lap_mat / rw_max_lambda - id_mat

        # For GCNConv
        # hat_L_rw = wid_D^{-1} * wid_A
        hat_rw_normd_lap_mat = np.matmul(wid_deg_mat_inv, wid_adj_mat)

        if mat_type == 'rw_normd_lap_mat':
            return rw_normd_lap_mat
        elif mat_type == 'wid_rw_normd_lap_mat':
            return wid_rw_normd_lap_mat
        elif mat_type == 'hat_rw_normd_lap_mat':
            return hat_rw_normd_lap_mat

# Utils

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

class Align(nn.Module):
    def __init__(self, c_in, c_out):
        super(Align, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        self.align_conv = nn.Conv2d(in_channels=c_in, out_channels=c_out, kernel_size=(1, 1))

    def forward(self, x):
        if self.c_in > self.c_out:
            x_align = self.align_conv(x)
        elif self.c_in < self.c_out:
            batch_size, c_in, timestep, n_vertex = x.shape
            x_align = torch.cat([x, torch.zeros([batch_size, self.c_out - self.c_in, timestep, n_vertex]).to(x)], dim=1)
        else:
            x_align = x
        
        return x_align

class CausalConv1d(nn.Conv1d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, enable_padding=False, dilation=1, groups=1, bias=True):
        if enable_padding == True:
            self.__padding = (kernel_size - 1) * dilation
        else:
            self.__padding = 0
        super(CausalConv1d, self).__init__(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=self.__padding, dilation=dilation, groups=groups, bias=bias)

    def forward(self, input):
        result = super(CausalConv1d, self).forward(input)
        if self.__padding != 0:
            return result[: , : , : -self.__padding]
        
        return result

class CausalConv2d(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, enable_padding=False, dilation=1, groups=1, bias=True):
        kernel_size = nn.modules.utils._pair(kernel_size)
        stride = nn.modules.utils._pair(stride)
        dilation = nn.modules.utils._pair(dilation)
        if enable_padding == True:
            self.__padding = [int((kernel_size[i] - 1) * dilation[i]) for i in range(len(kernel_size))]
        else:
            self.__padding = 0
        self.left_padding = nn.modules.utils._pair(self.__padding)
        super(CausalConv2d, self).__init__(in_channels, out_channels, kernel_size, stride=stride, padding=0, dilation=dilation, groups=groups, bias=bias)
        
    def forward(self, input):
        if self.__padding != 0:
            input = F.pad(input, (self.left_padding[1], 0, self.left_padding[0], 0))
        result = super(CausalConv2d, self).forward(input)

        return result

# Temporal Module

In [ ]:
class TemporalConvLayer(nn.Module):

    # Temporal Convolution Layer (GLU)
    #
    #        |-------------------------------| * residual connection *
    #        |                               |
    #        |    |--->--- casual conv ----- + -------|       
    # -------|----|                                   ⊙ ------>
    #             |--->--- casual conv --- sigmoid ---|                               
    #
    
    #param x: tensor, [batch_size, c_in, timestep, n_vertex]

    def __init__(self, Kt, c_in, c_out, n_vertex, act_func, enable_gated_act_func):
        super(TemporalConvLayer, self).__init__()
        self.Kt = Kt
        self.c_in = c_in
        self.c_out = c_out
        self.n_vertex = n_vertex
        self.act_func = act_func
        self.enable_gated_act_func = enable_gated_act_func
        self.align = Align(c_in, c_out)
        if enable_gated_act_func == True:
            self.causal_conv = CausalConv2d(in_channels=c_in, out_channels=2 * c_out, kernel_size=(Kt, 1), enable_padding=False, dilation=1)
        else:
            self.causal_conv = CausalConv2d(in_channels=c_in, out_channels=c_out, kernel_size=(Kt, 1), enable_padding=False, dilation=1)
        self.linear = nn.Linear(n_vertex, n_vertex)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()
        self.elu = nn.ELU()

    def forward(self, x):   
        x_in = self.align(x)[:, :, self.Kt - 1:, :]
        x_causal_conv = self.causal_conv(x)

        if self.enable_gated_act_func == True:
            x_p = x_causal_conv[:, : self.c_out, :, :]
            x_q = x_causal_conv[:, -self.c_out:, :, :]

            # Temporal Convolution Layer (GLU)
            if self.act_func == 'glu':
                # GLU was first purposed in
                # Language Modeling with Gated Convolutional Networks
                # https://arxiv.org/abs/1612.08083
                # Input tensor X was split by a certain dimension into tensor X_a and X_b
                # In original paper, GLU as Linear(X_a) ⊙ Sigmoid(Linear(X_b))
                # However, in PyTorch, GLU as X_a ⊙ Sigmoid(X_b)
                # https://pytorch.org/docs/master/nn.functional.html#torch.nn.functional.glu
                # Because in original paper, the representation of GLU and GTU is ambiguous
                # So, it is arguable which one version is correct

                # (x_p + x_in) ⊙ Sigmoid(x_q)
                x_glu = torch.mul((x_p + x_in), self.sigmoid(x_q))
                x_tc_out = x_glu

            # Temporal Convolution Layer (GTU)
            elif self.act_func == 'gtu':
                # Tanh(x_p + x_in) ⊙ Sigmoid(x_q)
                x_gtu = torch.mul(self.tanh(x_p + x_in), self.sigmoid(x_q))
                x_tc_out = x_gtu

            else:
                raise ValueError(f'ERROR: activation function {self.act_func} is not defined.')

        else:
            # Temporal Convolution Layer (Linear)
            if self.act_func == 'linear':
                x_linear = self.linear(x_causal_conv + x_in)
                x_tc_out = x_linear
            
            # Temporal Convolution Layer (Sigmoid)
            elif self.act_func == 'sigmoid':
                x_sigmoid = self.sigmoid(x_causal_conv + x_in)
                x_tc_out = x_sigmoid

            # Temporal Convolution Layer (Tanh)
            elif self.act_func == 'tanh':
                x_tanh = self.tanh(x_causal_conv + x_in)
                x_tc_out = x_tanh

            # Temporal Convolution Layer (ReLU)
            elif self.act_func == 'relu':
                x_relu = self.relu(x_causal_conv + x_in)
                x_tc_out = x_relu
        
            # Temporal Convolution Layer (LeakyReLU)
            elif self.act_func == 'leaky_relu':
                x_leaky_relu = self.leaky_relu(x_causal_conv + x_in)
                x_tc_out = x_leaky_relu

            # Temporal Convolution Layer (ELU)
            elif self.act_func == 'elu':
                x_elu = self.elu(x_causal_conv + x_in)
                x_tc_out = x_elu

            else:
                raise ValueError(f'ERROR: activation function {self.act_func} is not defined.')
        return x_tc_out

# Relation Aggregation

In [ ]:
class AttnAgg(nn.Module):
     def __init__(self, embed_dim): 
        super(AttnAgg, self).__init__()
        self.attn = nn.Linear(embed_dim, embed_dim)
        self.attn_relu = nn.ReLU()
        self.attn_weight = nn.Linear(embed_dim, 1)
        
     def forward(self, input):
        input = input.permute(0, 1, 3, 4, 2)
        K, batch_size, T, n_node, c_in = input.shape
        x_gc_first_attn = input.reshape(K, -1, c_in)
        attn = F.softmax(self.attn_weight(x_gc_first_attn).permute(1, 2, 0), -1)
        x_gc_second_attn = torch.bmm(attn, x_gc_first_attn.permute(1, 0, 2)).squeeze(1) # [B, 1, self.c_out]
        agg_output = x_gc_second_attn.reshape(batch_size, T, n_node, c_in).permute(0, 3, 1, 2)
        return agg_output

# Spatial Module

In [ ]:
class MGCNConv(nn.Module):
    def __init__(self, c_in, c_out, enable_bias, graph_conv_act_func, K=2):
        super(MGCNConv, self).__init__()
        self.c_in = c_in
        self.c_out = c_out
        self.enable_bias = enable_bias
        self.graph_conv_act_func = graph_conv_act_func
        # self.out = nn.Linear(c_in*K, c_out)
        self.relu = nn.ReLU()
        self.K = K
        self.weight = nn.Parameter(torch.cuda.FloatTensor(K, c_in, c_out))
        if enable_bias == True:
            self.bias = nn.Parameter(torch.cuda.FloatTensor(c_out))
        else:
            self.register_parameter('bias', None)
        self.initialize_parameters()

    def initialize_parameters(self):
        # For Sigmoid or Tanh
        if self.graph_conv_act_func == 'sigmoid' or self.graph_conv_act_func == 'tanh':
            init.xavier_uniform_(tensor=self.weight, gain=init.calculate_gain(self.graph_conv_act_func))

        # For ReLU, LeakyReLU, or ELU
        elif self.graph_conv_act_func == 'relu' or self.graph_conv_act_func == 'leaky_relu' \
            or self.graph_conv_act_func == 'elu':
            init.kaiming_uniform_(self.weight)

        if self.bias is not None:
            init.zeros_(self.bias)

    def forward(self, x, gcnconv_matrix):
        batch_size, c_in, T, n_vertex = x.shape
        n_matrix = gcnconv_matrix.shape[0]
        x_first_mul = torch.einsum('ab, cbd->cad', x.reshape(-1, c_in), self.weight).view(self.K, batch_size*T, n_vertex, -1) # [batch, c_in], [K, c_in, c_out] -> [K, batch, c_out]
        x_second_mul = torch.einsum('ecab,ecbd->ecad', gcnconv_matrix, x_first_mul) # [n_adj, B*T, n_node1, n_node2] * [n_adj, batch_size * T, n_node2, c_out] -> [n_adj, B*T, n_node1, c_out]
        if self.bias is not None:
            x_gcnconv = x_second_mul + self.bias
        else:
            x_gcnconv = x_second_mul
        x_gcnconv = x_gcnconv.view(n_matrix, batch_size, T, -1, self.c_out).permute(0, 1, 4, 2, 3) # [n_adj, batch_size, T, n_node1, c_out]
        return self.relu(x_gcnconv)

In [ ]:
class GraphConvLayer(nn.Module):
    def __init__(self, Ks, c_in, c_out, graph_conv_type, graph_conv_matrix, graph_conv_act_func, mode):
        super(GraphConvLayer, self).__init__()
        self.Ks = Ks
        self.c_in = c_in
        self.c_out = c_out
        self.align = Align(c_in, c_out)
        self.graph_conv_type = graph_conv_type
        self.graph_conv_matrix = graph_conv_matrix
        self.graph_conv_act_func = graph_conv_act_func
        self.enable_bias = True
        self.mode = mode

        self.intra_gcnconv = MGCNConv(c_out, c_out, self.enable_bias, graph_conv_act_func)
        self.inter_gcnconv = MGCNConv(c_out, c_out, self.enable_bias, graph_conv_act_func)
        self.rel_agg = AttnAgg(c_out)


    def forward(self, x, x_inter):
        x_in = self.align(x)
        batch_size, c_in, T, n_node1 = x_in.shape # n-vertex: n_node2
        # print(x_in.shape)

        x_inter_in = self.align(x_inter)
        batch_size, c_in, T, n_node2 = x_inter_in.shape # n-vertex: n_node2

        intra_adj = self.graph_conv_matrix[0].unsqueeze(1).repeat(1, batch_size*T, 1, 1) #[2, batch_size*T, n_node1, n_node2]
        inter_adj = self.graph_conv_matrix[1].unsqueeze(1).repeat(1, batch_size*T, 1, 1) # [2, batch_size*T, n_node1, n_node2]
        x_intra = self.intra_gcnconv(x_in, intra_adj)
        x_inter = self.inter_gcnconv(x_inter_in, inter_adj)

        x_gc_with_rc = torch.cat([x_intra, x_inter], 0)
        x_gc_with_rc = self.rel_agg(x_gc_with_rc)
        # x_gc_with_rc = torch.add(x_intra, x_inter)
        x_gc_out = torch.add(x_gc_with_rc, x_in)

        return x_gc_out

# ST-BLOCK & OUTPUT BLOCK

In [ ]:
class STConvBlock(nn.Module):
    # STConv Block contains 'TGTND' structure
    # T: Gated Temporal Convolution Layer (GLU or GTU)
    # G: Graph Convolution Layer (ChebConv or GCNConv)
    # T: Gated Temporal Convolution Layer (GLU or GTU)
    # N: Layer Normolization
    # D: Dropout

    def __init__(self, Kt, Ks, n_node1, n_node2, last_block_channel, channels, gated_act_func, graph_conv_type, graph_conv_matrix, drop_rate):
        super(STConvBlock, self).__init__()
        self.Kt = Kt
        self.Ks = Ks

        self.n_node1 = n_node1
        self.n_node2 = n_node2

        self.last_block_channel = last_block_channel
        self.channels = channels
        self.gated_act_func = gated_act_func
        self.enable_gated_act_func = True
        self.graph_conv_type = graph_conv_type
        self.graph_conv_matrix = graph_conv_matrix
        self.graph_conv_act_func = 'relu'
        self.drop_rate = drop_rate

        self.tmp_conv11 = TemporalConvLayer(Kt, last_block_channel, channels[0], n_node1, gated_act_func, self.enable_gated_act_func)
        self.graph_conv1 = GraphConvLayer(Ks, channels[0], channels[1], graph_conv_type, graph_conv_matrix[:2], self.graph_conv_act_func, 'subway')
        self.tmp_conv12 = TemporalConvLayer(Kt, channels[1], channels[2], n_node1, gated_act_func, self.enable_gated_act_func)

        self.tmp_conv21 = TemporalConvLayer(Kt, last_block_channel, channels[0], n_node2, gated_act_func, self.enable_gated_act_func)
        self.graph_conv2 = GraphConvLayer(Ks, channels[0], channels[1], graph_conv_type, graph_conv_matrix[2:], self.graph_conv_act_func, 'taxi')
        self.tmp_conv22 = TemporalConvLayer(Kt, channels[1], channels[2], n_node2, gated_act_func, self.enable_gated_act_func)
        
        self.tc1_ln = nn.LayerNorm([n_node1, channels[2]])
        self.tc2_ln = nn.LayerNorm([n_node2, channels[2]])

        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()
        self.elu = nn.ELU()
        self.do = nn.Dropout(p=drop_rate)

    def forward(self, x1, x2):
        # print('x1', x1.shape, 'x2', x2.shape)
        x_tmp_conv11 = self.tmp_conv11(x1)
        x_tmp_conv21 = self.tmp_conv21(x2)
        # print('x_tmp_conv11', x_tmp_conv11.shape)
        # print('x_tmp_conv21', x_tmp_conv21.shape)

        x_graph_conv1 = self.graph_conv1(x_tmp_conv11, x_tmp_conv21) # [n_adj, batch_size, T, n_node1, c_out]
        x_graph_conv2 = self.graph_conv2(x_tmp_conv21, x_tmp_conv11) # [n_adj, batch_size, T, n_node1, c_out]
        # print('x_graph_conv1', x_graph_conv1.shape)
        # print('x_graph_conv2', x_graph_conv2.shape)
        
        x_act_func1 = self.relu(x_graph_conv1)
        x_act_func2 = self.relu(x_graph_conv2)

        x_tmp_conv12 = self.tmp_conv12(x_act_func1)
        x_tmp_conv22 = self.tmp_conv22(x_act_func2)

        x_tc1_ln = self.tc1_ln(x_tmp_conv12.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)
        x_tc2_ln = self.tc2_ln(x_tmp_conv22.permute(0, 2, 3, 1)).permute(0, 3, 1, 2)

        x_do1 = self.do(x_tc1_ln)
        x_do2 = self.do(x_tc2_ln)

        return x_do1, x_do2

class OutputBlock(nn.Module):
    # Output block contains 'TNFF' structure
    # T: Gated Temporal Convolution Layer (GLU or GTU)
    # N: Layer Normolization
    # F: Fully-Connected Layer
    # F: Fully-Connected Layer

    def __init__(self, Ko, last_block_channel, channels, end_channel, n_vertex, gated_act_func, drop_rate):
        super(OutputBlock, self).__init__()
        self.Ko = Ko
        self.last_block_channel = last_block_channel
        self.channels = channels
        self.end_channel = end_channel
        self.n_vertex = n_vertex
        self.gated_act_func = gated_act_func
        self.enable_gated_act_func = True
        self.drop_rate = drop_rate
        self.tmp_conv1 = TemporalConvLayer(Ko, last_block_channel, channels[0], n_vertex, gated_act_func, self.enable_gated_act_func)
        self.fc1 = nn.Linear(channels[0], channels[1])
        self.fc2 = nn.Linear(channels[1], end_channel)
        self.tc1_ln = nn.LayerNorm([n_vertex, channels[0]])
        self.act_func = 'sigmoid'
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()
        self.elu = nn.ELU()
        self.do = nn.Dropout(p=self.drop_rate)

    def forward(self, x):
        x_tc1 = self.tmp_conv1(x)
        x_tc1_ln = self.tc1_ln(x_tc1.permute(0, 2, 3, 1))
        x_fc1 = self.fc1(x_tc1_ln)
        if self.act_func == 'sigmoid':
            x_act_func = self.sigmoid(x_fc1)
        elif self.act_func == 'tanh':
            x_act_func = self.tanh(x_fc1)
        elif self.act_func == 'relu':
            x_act_func = self.relu(x_fc1)
        elif self.act_func == 'leaky_relu':
            x_act_func = self.leaky_relu(x_fc1)
        elif self.act_func == 'elu':
            x_act_func = self.elu(x_fc1)
        x_fc2 = self.fc2(x_act_func).permute(0, 3, 1, 2)
        x_out = x_fc2

        return x_out

# Model

In [ ]:
class mySequential(nn.Sequential):
    def forward(self, *input):
        for module in self._modules.values():
            input = module(*input)
        return input

class STGCN_ChebConv(nn.Module):
    def __init__(self, Kt, Ks, blocks, T, n_node1, n_node2, gated_act_func, graph_conv_type, graph_conv_matrix, drop_rate):
        super(STGCN_ChebConv, self).__init__()
        self.n_node1 = n_node1
        self.n_node2 = n_node2
        modules = []

        # use to compute adaptive adj matrix

        # self.graph_conv_matrix = graph_conv_matrix
        for l in range(len(blocks) - 3):
            modules.append(STConvBlock(Kt, Ks, n_node1, n_node2, blocks[l][-1], blocks[l+1], gated_act_func, graph_conv_type, graph_conv_matrix, drop_rate))
        self.st_blocks = mySequential(*modules)

        # Ko = T - (len(blocks) - 3) * 2 * (Kt - 1)
        Ko = T - (len(blocks) - 3) * (Kt - 1)
        self.Ko = Ko
        print(Ko)
        if self.Ko > 1:
            self.output1 = OutputBlock(Ko, blocks[-3][-1], blocks[-2], blocks[-1][0], n_node1, gated_act_func, drop_rate)
            self.output2 = OutputBlock(Ko, blocks[-3][-1], blocks[-2], blocks[-1][0], n_node2, gated_act_func, drop_rate)
        elif self.Ko <= 1:
            self.fc1 = nn.Linear(blocks[-3][-1], blocks[-2][0])
            self.fc2 = nn.Linear(blocks[-2][0], blocks[-1][0])
            self.act_func = 'sigmoid'
            self.sigmoid = nn.Sigmoid()
            self.tanh = nn.Tanh()
            self.relu = nn.ReLU()
            self.leaky_relu = nn.LeakyReLU()
            self.elu = nn.ELU()
            self.do = nn.Dropout(p=drop_rate)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)   # batch_size, c_in, T, n_vertex 
        x1, x2 = x[:, :, :, :self.n_node1], x[:, :, :, self.n_node1:]

        x_st1, x_st2 = self.st_blocks(x1, x2)
        
        x_out1 = self.output1(x_st1).permute(0, 2, 3, 1)
        x_out2 = self.output2(x_st2).permute(0, 2, 3, 1)
        
        return x_out1, x_out2

# Data Container

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import h5py

class DataInput(object):
    def __init__(self, data_dir1, data_dir2, norm_opt=True):
        self.data_dir1 = data_dir1
        self.data_dir2 = data_dir2
        self.min1, self.max1 = 0, 0
        self.min2, self.max2 = 0, 0
        self.n_node1, self.n_node2 = 0, 0
        self.norm_opt = norm_opt

    def load_data(self):
        print('Loading data...')
        dataset = dict()
        data1 = np.load(self.data_dir1)
        print(data1.shape)
        self.min1, self.max1, self.n_node1 = data1.min(), data1.max(), data1.shape[1]
        print(self.min1, self.max1, self.n_node1)
        data1 = self.minmax_normalize(data1, self.min1, self.max1) if self.norm_opt else data
        
        data2 = np.load(self.data_dir2)
        print(data2.shape)
        self.min2, self.max2, self.n_node2 = data2.min(), data2.max(), data2.shape[1]
        print(self.min2, self.max2, self.n_node2)
        data2 = self.minmax_normalize(data2, self.min2, self.max2) if self.norm_opt else data

        dataset['concat'] = np.concatenate([data1, data2], 1)
        return dataset

    def minmax_normalize(self, x:np.array, min, max):
        x = (x - min) / (max - min)
        x = 2 * x - 1
        return x

    def minmax_denormalize(self, x:np.array, min, max):
        x = (x + 1)/2
        x = (max - min) * x + min
        return x

class TaxiDataset(Dataset):
    '''
        inputs: history obs: short-term seq | daily seq | weekly seq (B, seq, N, C)
        output: y_t+1 target (B, N, C)
        mode: one in [train, validate, test]
        mode_len: {train, validate, test}
    '''
    def __init__(self, device:str, inputs:dict, output:np.array, mode:str, mode_len:dict, start_idx:int):
        self.device = device
        self.mode = mode
        self.mode_len = mode_len
        self.start_idx = start_idx  # train_start idx
        self.inputs, self.output = self.prepare_xy(inputs, output)

    def __len__(self):
        return self.mode_len[self.mode]

    def __getitem__(self, item):
        return self.inputs['x_seq'][item], self.output[item]

    # to solve tomorrow morning: I think there is some problem with the start_idx it sets. It sets it as day rather than time intervals.
    def prepare_xy(self, inputs:dict, output:np.array):
        if self.mode == 'train':
            pass
        elif self.mode == 'validate':
            self.start_idx += self.mode_len['train']
        else:       # test
            self.start_idx += (self.mode_len['train'] + self.mode_len['validate'])

        obs = []
        for kw in ['weekly', 'daily', 'serial']:
            if len(inputs[kw].shape) != 2:      # dim=2 for empty seq
                obs.append(inputs[kw])
        x_seq = np.concatenate(obs, axis=1)   # concatenate timeslices to one seq
        x = dict()
        x['x_seq'] = torch.from_numpy(x_seq[self.start_idx :  (self.start_idx + self.mode_len[self.mode])]).float().to(self.device)
        y = torch.from_numpy(output[self.start_idx : self.start_idx + self.mode_len[self.mode]]).float().to(self.device)
        return x, y

class DataGenerator(object):
    def __init__(self, dt:int, obs_len:tuple, pre_horizon:int):
        self.day_timesteps = int(24*dt)
        self.serial_len, self.daily_len, self.weekly_len = obs_len
        self.pre_horizon = pre_horizon
        self.start_idx = 0
        self.mode_len = dict()
        # self.start_idx, self.mode_len = self.date2len(data_len=data_len)

    def date2len(self, data_len:int):
        d_len = data_len - self.pre_horizon + 1
        day_count = d_len//self.day_timesteps
        split_line1 = int(day_count * 0.6) * self.day_timesteps
        split_line2 = int(day_count * 0.8) * self.day_timesteps
        return 0, {'train':split_line1, 'validate':split_line2-split_line1, 'test':d_len-split_line2}

    def get_data_loader(self, data:dict, batch_size:int, device:str):
        feat_dict = dict()
        feat_dict['serial'], feat_dict['daily'], feat_dict['weekly'], output = self.get_feats(data['concat'])

        print(feat_dict['serial'].shape)
        data_len = feat_dict['serial'].shape[0]
        self.start_idx, self.mode_len = self.date2len(data_len)
        data_loader = dict() # data_loader for [train, validate, test]
        for mode in ['train', 'validate', 'test']:
            dataset = TaxiDataset(device=device, inputs=feat_dict, output=output,
                                  mode=mode, mode_len=self.mode_len, start_idx=self.start_idx)
            data_loader[mode] = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)
        return data_loader

    def get_feats(self, data:np.array):
        serial, daily, weekly, y = [], [], [], []
        start_idx = max(self.serial_len, self.daily_len*self.day_timesteps, self.weekly_len*self.day_timesteps * 7)
        end_idx = data.shape[0] - self.pre_horizon + 1
        for i in range(start_idx, end_idx):
            serial.append(data[i-self.serial_len : i])
            daily.append(self.get_periodic_skip_seq(data, i, 'daily'))
            weekly.append(self.get_periodic_skip_seq(data, i, 'weekly'))
            y.append(data[i:i+self.pre_horizon])
        return np.array(serial), np.array(daily), np.array(weekly), np.array(y)

    def get_periodic_skip_seq(self, data:np.array, idx:int, p:str):
        p_seq = list()
        if p == 'daily':
            p_steps = self.daily_len * self.day_timesteps
            for d in range(1, self.daily_len+1):
                p_seq.append(data[idx - p_steps*d])
        else:   # weekly
            p_steps = self.weekly_len * self.day_timesteps * 7
            for w in range(1, self.weekly_len+1):
                p_seq.append(data[idx - p_steps*w])
        p_seq = p_seq[::-1]     # inverse order
        return np.array(p_seq)

# Model Trainer

In [ ]:
import datetime
class ModelTrainer(object):
    def __init__(self, model_name: str, model:nn.Module, loss:nn.Module, optimizer, lr:float, wd:float, n_epochs:int, n_node1: int, n_node2: int, a=0.5):
        self.model = model
        self.model_name = model_name
        self.criterion = loss
        self.optimizer = optimizer(params=self.model.parameters(), lr=lr, weight_decay=wd)
        self.n_epochs = n_epochs
        self.n_node1 = n_node1
        self.n_node2 = n_node2
        self.a = a

    def train(self, data_loader:dict, modes:list, model_dir:str, data_class, early_stopper=100):
        checkpoint = {'epoch':0, 'state_dict':self.model.state_dict()}
        val_loss = np.inf

        start_time = datetime.datetime.now()
        for epoch in range(1, self.n_epochs+1):

            running_loss = {mode:0.0 for mode in modes}
            for mode in modes:
                if mode == 'train':
                    self.model.train()
                else:
                    self.model.eval()

                step = 0
                for x, y_true in data_loader[mode]:
                    # print('x', x.size(), 'y_true', y_true.size())
                    with torch.set_grad_enabled(mode = mode=='train'):
                        y_pred1, y_pred2 = self.model(x)
                        y_true1, y_true2 = y_true[:, :, :self.n_node1, :], y_true[:, :, self.n_node1:, :]
                        loss = self.a * self.criterion(y_pred1, y_true1) + (1-self.a) * self.criterion(y_pred2, y_true2)
                        if mode == 'train':
                            self.optimizer.zero_grad()
                            loss.backward()
                            self.optimizer.step()
                    running_loss[mode] += loss * y_true.shape[0]
                    step += y_true.shape[0]

                # epoch end
                if mode == 'validate':
                    if running_loss[mode]/step <= val_loss:
                        # print(f'Epoch {epoch}, Val_loss drops from {val_loss:.5} to {running_loss[mode]/step:.5}. '
                        #       f'Update model checkpoint..')
                        val_loss = running_loss[mode]/step
                        checkpoint.update(epoch=epoch, state_dict=self.model.state_dict())
                        torch.save(checkpoint, model_dir + f'/{self.model_name}_best_model.pkl')
                        early_stopper = 100
                    else:
                        # print(f'Epoch {epoch}, Val_loss does not improve from {val_loss:.5}.')
                        early_stopper -= 1
                        if early_stopper == 0:
                            print(f'Early stopping at epoch {epoch}..')
                            return
            if epoch % 50 == 0: 
                self.test(epoch=epoch, data_loader=data_loader, modes=['train', 'test'], model_dir=model_dir, data_class=data_in)
        # print('Training ends at: ', time.ctime())
        torch.save(checkpoint, model_dir + f'/{self.model_name}_best_model.pkl')
        print('training', datetime.datetime.now() - start_time)
        return

    # since cuda always runs out of memory, we will conduct test on cpu.
    def test(self, epoch, data_loader:dict, modes:list, model_dir:str, data_class):

        saved_checkpoint = torch.load(model_dir + f'/{self.model_name}_best_model.pkl')
        self.model.load_state_dict(saved_checkpoint['state_dict'])
        self.model.eval()

        # print('Testing starts at: ', time.ctime())
        running_loss = {mode: 0.0 for mode in modes}
        start_time = datetime.datetime.now()
        for mode in modes:
            ground_truth1, prediction1 = list(), list()
            ground_truth2, prediction2 = list(), list()
            for x, y_true in data_loader[mode]:
                y_pred1, y_pred2 = self.model(x)
                y_true1, y_true2 = y_true[:, :, :self.n_node1, :], y_true[:, :, self.n_node1:, :]

                ground_truth1.append(y_true1.cpu().detach().numpy())
                prediction1.append(y_pred1.cpu().detach().numpy())

                ground_truth2.append(y_true2.cpu().detach().numpy())
                prediction2.append(y_pred2.cpu().detach().numpy())

            ground_truth1 = data_class.minmax_denormalize(np.concatenate(ground_truth1, axis=0), data_class.min1, data_class.max1)
            prediction1 = data_class.minmax_denormalize(np.concatenate(prediction1, axis=0), data_class.min1, data_class.max1)
            print('Mode1', f'{epoch}{mode} RMSE: ', self.RMSE(prediction1, ground_truth1), 'MAE:', self.MAE(prediction1, ground_truth1), 'MAPE:', self.MAPE(prediction1, ground_truth1), 'PCC:', self.PCC(prediction1, ground_truth1))

            ground_truth2 = data_class.minmax_denormalize(np.concatenate(ground_truth2, axis=0), data_class.min2, data_class.max2)
            prediction2 = data_class.minmax_denormalize(np.concatenate(prediction2, axis=0), data_class.min2, data_class.max2)
            print('Mode2', f'{epoch}{mode} RMSE: ', self.RMSE(prediction2, ground_truth2), 'MAE:', self.MAE(prediction2, ground_truth2), 'MAPE:', self.MAPE(prediction2, ground_truth2), 'PCC:', self.PCC(prediction2, ground_truth2))
        return

    @staticmethod
    def MSE(y_pred:np.array, y_true:np.array):
        return np.mean(np.square(y_pred - y_true))
    @staticmethod
    def RMSE(y_pred:np.array, y_true:np.array):
        return np.sqrt(np.mean(np.square(y_pred - y_true)))
    @staticmethod
    def MAE(y_pred:np.array, y_true:np.array):
        return np.mean(np.abs(y_pred - y_true))
    @staticmethod
    def MAPE(y_pred:np.array, y_true:np.array, epsilon=1e-0):   # zero division
        return np.mean(np.abs(y_pred - y_true) / (y_true + epsilon))
    @staticmethod
    def PCC(y_pred:np.array, y_true:np.array):
        return np.corrcoef(y_pred.flatten(), y_true.flatten())[0,1]

# Model Training and Testing

In [ ]:
"""load demand data"""
import numpy as np
M_adj = 1 # num static adjs

data_path1 = "../data/metro_demand_4h.npy"
data_path2 = "../data/FR_demand_4h.npy"

data_in = DataInput(data_dir1=data_path1, data_dir2=data_path2, norm_opt=True)
data = data_in.load_data()
n_node1, n_node2 = data_in.n_node1, data_in.n_node2
print(data_in.n_node1, data_in.n_node2)

In [ ]:
"""load adj data"""
bb_adj_path = "../data/relationship/geo/Metro_adj.npy"
tt_adj_path = "../data/relationship/geo/Unit_adj.npy"
bt_adj_path =  "../data/relationship/geo/Metro_Unit_adj.npy"

A = []
A.append(np.load(bb_adj_path))
A.append(np.load(bt_adj_path))
A.append(np.load(tt_adj_path))
A.append(np.transpose(np.load(bt_adj_path)))
assert len(A) == 4

bb_pat_path = "../data/relationship/pattern/Metro_adj.npy"
tt_pat_path = "../data/relationship/pattern/Unit_adj.npy"
bt_pat_path =  "../data/relationship/pattern/Metro_Unit_adj.npy"

A_pat = []
A_pat.append(np.load(bb_pat_path))
A_pat.append(np.load(bt_pat_path))
A_pat.append(np.load(tt_pat_path))
A_pat.append(np.transpose(np.load(bt_pat_path)))
assert len(A_pat) == 4

mat_type = 'hat_rw_normd_lap_mat'
gcnconv_matrix_list = []
for adj_mat, pat_mat in zip(A, A_pat):
    A_adj = torch.from_numpy((calculate_random_walk_matrix(adj_mat.T).T).astype('float32')).to(device) # This is for D-GCN
    A_pat = torch.from_numpy((calculate_random_walk_matrix(pat_mat.T).T).astype('float32')).to(device)
    A_h = torch.cat([A_adj.unsqueeze(0), A_pat.unsqueeze(0)], 0)
    gcnconv_matrix_list.append(A_h)
    print(adj_mat.shape)
    print(A_h.shape)

In [ ]:
"""prepare data for model training and testing"""
dt = 0.25

obs_len = (6, 0, 0) # short-term/daily/weekly observations

pre_horizon = 1

batch_size = 32

# data_generator = DataGenerator(dt=dt, obs_len=obs_len, train_test_dates=dates, val_ratio=0.2, test_ratio=0.2)
data_generator = DataGenerator(dt=dt, obs_len=obs_len, pre_horizon=pre_horizon)

data_loader = data_generator.get_data_loader(data=data, batch_size=batch_size, device=device)

In [ ]:
"""hyperparameter settings"""
Ks = 2
Kt = 2

epoch = 500

learn_rate, weight_decay = 2e-3, 1e-5

dropout = 0.3

stblock_num = 2

n_his = sum(obs_len)
Ko = n_his - (Kt - 1) * 2 * stblock_num

blocks = []
blocks.append([2])
for l in range(stblock_num):
    blocks.append([64, 16, 64])
if Ko == 0:
    blocks.append([128])
elif Ko > 0:
    blocks.append([128, 128])
blocks.append([2])

gated_act_func = 'glu' # 'glu' for GLU, 'gtu' for GTU (temporal convolution)

graph_conv_type = 'gcnconv' # 'chebconv' for STGCN_ChebConv, 'gcnconv' for STGCN_GCNConv

loss = nn.MSELoss(reduction='sum')
   
optimizer = optim.Adam

In [ ]:
for i in range(10):

    model_name = 'STMRGNN_%d'%i

    model = STGCN_ChebConv(Kt, Ks, blocks, n_his, n_node1, n_node2, gated_act_func, graph_conv_type, gcnconv_matrix_list, dropout).to(device)

    trainer = ModelTrainer(model_name=model_name, model=model, loss=loss, optimizer=optimizer, lr=learn_rate, wd=weight_decay, n_epochs=epoch, n_node1=n_node1, n_node2=n_node2, a=0.5)

    model_dir = "../model"

    trainer.train(data_loader=data_loader, modes=['train', 'validate'], model_dir=model_dir, data_class=data_in)

    trainer.test(epoch=i, data_loader=data_loader, modes=['test'], model_dir=model_dir, data_class=data_in)